<a href="https://githubtocolab.com/kaust-halo/geeet/blob/master/examples/notebooks/01_geeet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Simple geeet demonstration

This notebook demonstrates the use of ET models in `geeet` with a minimum working (toy) example. It demonstrates the hybrid use of geeet models, i.e. using `ee.Images` and numerical data using the `geeet.tseb` and `geeet.ptjpl` modules as examples. We also show how to map these functions to an `ee.ImageCollection`. 


In [1]:
# Use the following line to install geeet if needed:
#!pip install git+https://github.com/kaust-halo/geeet
import ee
#ee.Authenticate() # Uncomment if using Google Colab or first time using EE on this device. 
ee.Initialize()

## Data setup 

Here we setup some numerical data that will be used as inputs to run the ET models.

In [2]:
## Location and time
lonlat=[38.25, 30] 
geom = ee.Geometry.Point(lonlat[0], lonlat[1])
region = geom.buffer(10000) # 10-km radius around the point 

# Inputs (n.b. not all are required for each model)
Ta = 25+273.15 # Air temperature, in Kelvin
RH = 25 # Relative humidity (%)
P = 95500 # Surface pressure, in Pascals 
U = 2 # Wind speed, in m/s
Tr = 26+273.15 # Radiometric (land surface) temperature, in Kelvin
NDVI = 0.75 # Normalized Difference Vegetation Index
lai = 2.408 # Leaf area index, in m2/m2 
fc = 0.7 # fractional vegetation cover 
fapar_max = 0.73 # Maximum fraction of photosynthetically active radiation 
Rn = 487.87 # Net radiation, in W/m2
Sdn = 745 # Shortwave radiation, in W/m2
Ldn = 345 # Longwave radiation, in W/m2
albedo = 0.22 
G = 59.17 # Soil heat flux, in W/m2
doy = 1 # Day of year 
Time = 11 # time of observation
zU = 10 # Wind speed observation height 
zT = 2 # Air temperature observation height
AlphaPT = 1.26 # Priestly-Taylor coefficient 
Leaf_width = 0.1 # average leaf size, in m
Vza = 0 # Viewing zenith angle, in degrees

## Two-source Energy Balance (TSEB) model

First we setup an image containing the necessary inputs to run the TSEB model and then call the `tseb_series` function to build the model. The result is an `ee.Image`, however note that nothing has yet computed until you actually request some data. 

In [3]:
sample_tseb_inputs = ee.Image(Tr).addBands(ee.Image(Ta)).addBands(ee.Image(U))\
.addBands(ee.Image(P)).addBands(ee.Image(Sdn)).addBands(ee.Image(Ldn))\
.addBands(ee.Image(NDVI)).addBands(ee.Image(albedo))
sample_tseb_inputs = sample_tseb_inputs.rename(['radiometric_temperature', 'air_temperature',\
    'wind_speed', 'surface_pressure', 'solar_radiation', 'thermal_radiation',\
    'NDVI', 'albedo']).set({'doy':doy, 'time':Time, 'viewing_zenith':Vza})
sample_tseb_inputs = sample_tseb_inputs.clip(region)

# Setup the TSEB model using the ee.Image defined above:
from geeet.tseb import tseb_series
et_tseb = tseb_series(img = sample_tseb_inputs, zU = zU, zT = zT)  # et_tseb is now an ee.Image, but no computations yet

In the next cell, we request data at a specific point (the lon/lat point we defined above) at a specific scale (100 m in this case):

In [4]:
# Actually compute the model to retrieve data over one point at a scale of 100 meters:
et_tseb_pix = et_tseb.reduceRegion(reducer=ee.Reducer.mean(), geometry=geom, scale=100).getInfo()

In the next cell we run the same function (`tseb_series`) using the numerical data directly (no GEE). We then compare the outputs obtained from the GEE and the local runs:

In [8]:
from geeet.tseb import tseb_series
out = tseb_series(\
    Tr = Tr, Alb = albedo, NDVI = NDVI, P = P, Ta = Ta, U = U, \
    Sdn = Sdn, Ldn = Ldn, doy = doy, time = Time, Vza = Vza,\
    longitude = lonlat[0], latitude = lonlat[1], zU = zU, zT = zT)
et_tseb_out = dict(zip(out,[out.get(x)[0] for x in out])) 

print('                                                GEE   |  local')
print('Latent heat flux (W/m2):                      {0:7.3f} | {1:7.3f}'.format(et_tseb_pix['LE']   ,et_tseb_out['LE']))
print('Latent heat flux (W/m2) from canopy source:   {0:7.3f} | {1:7.3f}'.format(et_tseb_pix['LEc']  ,et_tseb_out['LEc']))
print('Latent heat flux (W/m2) from soil source:     {0:7.3f} | {1:7.3f}'.format(et_tseb_pix['LEs']  ,et_tseb_out['LEs']))
print('Sensible heat flux (W/m2) from canopy source: {0:7.3f} | {1:7.3f}'.format(et_tseb_pix['Hc']   ,et_tseb_out['Hc']))
print('Sensible heat flux (W/m2) from soil source:   {0:7.3f} | {1:7.3f}'.format(et_tseb_pix['Hs']   ,et_tseb_out['Hs']))
print('Ground heat flux (W/m2):                      {0:7.3f} | {1:7.3f}'.format(et_tseb_pix['G']    ,et_tseb_out['G']))
print('Net radiation (W/m2):                         {0:7.3f} | {1:7.3f}'.format(et_tseb_pix['Rn']   ,et_tseb_out['Rn']))
print('Net radiation (W/m2) for the canopy source:   {0:7.3f} | {1:7.3f}'.format(et_tseb_pix['Rnc']  ,et_tseb_out['Rnc']))
print('Net radiation (W/m2) for the soil source:     {0:7.3f} | {1:7.3f}'.format(et_tseb_pix['Rns']  ,et_tseb_out['Rns']))
print('Energy balance closure (W/m2):                {0:7.3f} | {1:7.3f}'.format(\
    sum(list(map(et_tseb_pix.get,['LEc', 'LEs', 'Hc', 'Hs','G'])), -et_tseb_pix['Rn']),\
    sum(list(map(et_tseb_out.get,['LEc', 'LEs', 'Hc', 'Hs','G'])), -et_tseb_out['Rn'])))
print('Temperature (C) of the canopy source:         {0:7.3f} | {1:7.3f}'.format(et_tseb_pix['Tc']-273.15, et_tseb_out['Tc']-273.15)) 
print('Temperature (C) of the soil source:           {0:7.3f} | {1:7.3f}'.format(et_tseb_pix['Ts']-273.15, et_tseb_out['Ts']-273.15)) 
print('Resistance Ra (s/m):                          {0:7.3f} | {1:7.3f}'.format(et_tseb_pix['Ra']   ,et_tseb_out['Ra']))
print('Resistance Rs (s/m):                          {0:7.3f} | {1:7.3f}'.format(et_tseb_pix['Rs']   ,et_tseb_out['Rs']))
print('Resistance Rx (s/m):                          {0:7.3f} | {1:7.3f}'.format(et_tseb_pix['Rx']   ,et_tseb_out['Rx']))

                                                GEE   |  local
Latent heat flux (W/m2):                      410.254 | 410.254
Latent heat flux (W/m2) from canopy source:   296.981 | 296.982
Latent heat flux (W/m2) from soil source:     113.273 | 113.272
Sensible heat flux (W/m2) from canopy source:  18.997 |  18.997
Sensible heat flux (W/m2) from soil source:    10.794 |  10.794
Ground heat flux (W/m2):                       47.832 |  47.832
Net radiation (W/m2):                         487.877 | 487.877
Net radiation (W/m2) for the canopy source:   315.978 | 315.979
Net radiation (W/m2) for the soil source:     171.899 | 171.899
Energy balance closure (W/m2):                  0.000 |   0.000
Temperature (C) of the canopy source:          25.708 |  25.708
Temperature (C) of the soil source:            26.682 |  26.682
Resistance Ra (s/m):                           41.957 |  41.957
Resistance Rs (s/m):                          133.587 | 133.587
Resistance Rx (s/m):                     

## PT-JPL model

First we setup an image containing the necessary inputs to run the PT-JPL model and then call the `ptjpl_arid` function to build the model. The result is an `ee.Image`, however note that nothing has yet computed until you actually request some data. 

In [6]:
sample_ptjpl_inputs = ee.Image(Ta).addBands(ee.Image(RH))\
.addBands(ee.Image(P)).addBands(ee.Image(Rn))\
.addBands(ee.Image(NDVI)).addBands(ee.Image(fapar_max))
sample_ptjpl_inputs = sample_ptjpl_inputs.rename(['air_temperature',\
    'relative_humidity', 'surface_pressure', 'net_radiation',\
    'NDVI', 'fapar_max']).set({'doy':doy, 'time':Time})
sample_ptjpl_inputs = sample_ptjpl_inputs.clip(region)

# Setup the PTJPL model using the ee.Image defined above:
from geeet.ptjpl import ptjpl_arid
et_ptjpl = ptjpl_arid(sample_ptjpl_inputs)  # et_ptjpl is now an ee.Image, but no computations yet

In the next cell, we request data at a specific point (the lon/lat point we defined above) at a specific scale (100 m in this case):

In [7]:
# Actually compute the model to retrieve data over one point at a scale of 100 meters:
et_ptjpl_pix = et_ptjpl.reduceRegion(reducer=ee.Reducer.mean(), geometry=geom, scale=100).getInfo()

In the next cell we run the same function (`ptjpl_arid`) using the numerical data directly (no GEE). We then compare the outputs to those obtained in the previous cell.

In [8]:
# Run PTJPL model locally
et_ptjpl_out = ptjpl_arid(Ta = Ta, \
    P = P, NDVI = NDVI, F_aparmax=fapar_max, \
    Rn = Rn, RH = RH, doy = doy, time = Time, \
    longitude = lonlat[0])
# Print flux comparison
print('                                                GEE    |  local')
print('Latent heat flux (W/m2):                       {0:7.3f} | {1:7.3f}'.format(et_ptjpl_pix['LE']   ,et_ptjpl_out['LE']))
print('Latent heat flux (W/m2) from canopy source:    {0:7.3f} | {1:7.3f}'.format(et_ptjpl_pix['LEc']  ,et_ptjpl_out['LEc']))
print('Latent heat flux (W/m2) from soil source:      {0:7.3f} | {1:7.3f}'.format(et_ptjpl_pix['LEs']  ,et_ptjpl_out['LEs']))
print('Latent heat flux (W/m2) intercepted at canopy: {0:7.3f} | {1:7.3f}'.format(et_ptjpl_pix['LEi']  ,et_ptjpl_out['LEi']))
print('Sensible heat flux (W/m2):                     {0:7.3f} | {1:7.3f}'.format(et_ptjpl_pix['H']    ,et_ptjpl_out['H']))
print('Ground heat flux (W/m2):                       {0:7.3f} | {1:7.3f}'.format(et_ptjpl_pix['G']    ,et_ptjpl_out['G']))
print('Net radiation (W/m2):                          {0:7.3f} | {1:7.3f}'.format(et_ptjpl_pix['Rn']   ,et_ptjpl_out['Rn']))

                                                GEE    |  local
Latent heat flux (W/m2):                       327.255 | 327.255
Latent heat flux (W/m2) from canopy source:    322.688 | 322.688
Latent heat flux (W/m2) from soil source:        3.198 |   3.198
Latent heat flux (W/m2) intercepted at canopy:   1.369 |   1.369
Sensible heat flux (W/m2):                     128.604 | 128.604
Ground heat flux (W/m2):                        32.011 |  32.011
Net radiation (W/m2):                          487.870 | 487.870


n.b. the difference in ground heat flux between TSEB and PT-JPL is due to a different partitioning of the net radiation to the soil/canopy. Specifically, TSEB uses the solar zenith as an additional parameter inside the exponential factor: $\frac{1}{\sqrt{2\cos\theta}}$ where $\theta$ is the solar zenith. PT-JPL does not use this factor. For more information, see the `geeet.solar.compute_Rns` function. Further comparison of the ET models is beyond the scope of this notebook. 

## Mapping geeet models to an ee.ImageCollection

Here we show how to map `geeet` models to an `ee.ImageCollection`. Again, nothing is actually computed until we request some outputs, which we show how to do for a simple geometry (a point) using a reducer. 

In [9]:
# Simple image collection consisting of two images:
sample_collection = ee.ImageCollection([sample_ptjpl_inputs.set('doy',10), sample_ptjpl_inputs.set('doy',50)])
sample_collection_output = sample_collection.map(ptjpl_arid)

Here's one way to retrieve all the data over a specific region (e.g. an `ee.Geometry`): first define a function using the desired `ee.Reducer`, and return the data as a feature, then map this function over the output image collection. After this, you could export the feature collection using `ee.batch.Export.table.toDrive` (or `.toAsset`) (recommended way to export data if it is a large amount). For the purpose of this notebook, we can use `.getInfo()` to retrieve the data directly.  

In [10]:
# Function using a reducer to get the mean value of each band in the defined region.
def reduce_et(img):
    doy = img.get('doy')  
    img_mean = img.reduceRegion(ee.Reducer.mean(), geometry=geom, scale=100)
    feature = ee.Feature(geom, img_mean)
    feature = feature.set('doy', doy)
    return feature

sample_collection_output_pix  = sample_collection_output.map(reduce_et)
#export_task = ee.batch.Export.table.toDrive(collection = sample_collection_output_pix, description = 'ExportTable') # Export the results to drive
#print(sample_collection_output_pix.toList(count=2).getInfo())  # Retrieve and print the results here in JSON format. 

In [11]:
# Retrieve the data in json format
json_data = sample_collection_output_pix.toList(count=2).getInfo()  # This will be slow (computations are now 
                                                        # actually done in the Google Earth Engine servers and
                                                        # then sent here. )
# Create a list of dictionaries with the data:
list_dict = [x['properties'] for x in json_data]
# Transform the list of dictionaries to a dictionary of lists:
data_dict = {key: [d[key] for d in list_dict] for key in list_dict[0]}
print(data_dict)

{'G': [32.32386328885607, 32.80950191529322], 'H': [128.30272396808647, 127.83579140618627], 'LE': [327.2434127430575, 327.22470667852053], 'LEc': [322.6884951724099, 322.6884951724099], 'LEi': [1.3688096215331893, 1.3688096215331893], 'LEs': [3.186107949114378, 3.1674018845774485], 'NDVI': [0.75, 0.75], 'Rn': [487.87, 487.87], 'air_temperature': [298.15, 298.15], 'doy': [10, 50], 'fapar_max': [0.73, 0.73], 'net_radiation': [487.87, 487.87], 'relative_humidity': [25, 25], 'surface_pressure': [95500, 95500]}


In [12]:
# You could also now create a pandas dataframe with the data:
try:
    import pandas as pd       
    data_frame = pd.DataFrame(data_dict).set_index('doy')
    data_frame['EB closure'] = data_frame.apply(lambda x: x.G+x.H+x.LE-x.Rn, axis=1)
    print(data_frame)
except:
    pass

             G           H          LE         LEc      LEi       LEs  NDVI  \
doy                                                                           
10   32.323863  128.302724  327.243413  322.688495  1.36881  3.186108  0.75   
50   32.809502  127.835791  327.224707  322.688495  1.36881  3.167402  0.75   

         Rn  air_temperature  fapar_max  net_radiation  relative_humidity  \
doy                                                                         
10   487.87           298.15       0.73         487.87                 25   
50   487.87           298.15       0.73         487.87                 25   

     surface_pressure  EB closure  
doy                                
10              95500         0.0  
50              95500         0.0  
